# Home Work 3. Data Visualisation.

In [42]:
import pandas as pd
import numpy as np
import folium 
from lxml import etree
import json
import requests
from lxml import etree
import utils  # We defined  functions to be used in this  seperate file
from geopy.geocoders import Nominatim
import geocoder


%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Importing and Preprocessing of Data .

In [5]:
path = 'Data/P3_GrantExport.csv'

#We begin by importing the whole CSV thanks to the pandas library.

data = pd.read_csv(path,sep = ';')

In [6]:

data.head(100)

,"﻿""Project Number""",Project Title,Project Title English,Responsible Applicant,Funding Instrument,Funding Instrument Hierarchy,Institution,University,Discipline Number,Discipline Name,Discipline Name Hierarchy,Start Date,End Date,Approved Amount,Keywords
0,1,Schlussband (Bd. VI) der Jacob Burckhardt-Biog...,NaN,Kaegi Werner,Project funding (Div. I-III),Project funding,NaN,Nicht zuteilbar - NA,10302,Swiss history,Human and Social Sciences;Theology & religious...,01.10.1975,30.09.1976,11619.00,NaN
1,4,Batterie de tests à l'usage des enseignants po...,NaN,Massarenti Léonard,Project funding (Div. I-III),Project funding,Faculté de Psychologie et des Sciences de l'Ed...,Université de Genève - GE,10104,Educational science and Pedagogy,"Human and Social Sciences;Psychology, educatio...",01.10.1975,30.09.1976,41022.00,NaN
2,5,"Kritische Erstausgabe der ""Evidentiae contra D...",NaN,Kommission für das Corpus philosophorum medii ...,Project funding (Div. I-III),Project funding,Kommission für das Corpus philosophorum medii ...,"NPO (Biblioth., Museen, Verwalt.) - NPO",10101,Philosophy,Human and Social Sciences;Linguistics and lite...,01.03.1976,28.02.1985,79732.00,NaN
3,6,Katalog der datierten Handschriften in der Sch...,NaN,Burckhardt Max,Project funding (Div. I-III),Project funding,Abt. Handschriften und Alte Drucke Bibliothek ...,Universität Basel - BS,10302,Swiss history,Human and Social Sciences;Theology & religious...,01.10.1975,30.09.1976,52627.00,NaN
4,7,Wissenschaftliche Mitarbeit am Thesaurus Lingu...,NaN,Schweiz. Thesauruskommission,Project funding (Div. I-III),Project funding,Schweiz. Thesauruskommission,"NPO (Biblioth., Museen, Verwalt.) - NPO",10303,Ancient history and Classical studies,Human and Social Sciences;Theology & religious...,01.01.1976,30.04.1978,120042.00,NaN
5,8,Die schweizerische Wirtschaftspolitik seit dem...,NaN,Kleinewefers Henner,Project funding (Div. I-III),Project funding,"Séminaire de politique économique, d'économie ...",Université de Fribourg - FR,10203,Economics,"Human and Social Sciences;Economics, law",01.01.1976,31.12.1978,53009.00,NaN
6,9,Theologische Forschungen zur Oekumene (Studien...,NaN,Stirnimann Heinrich,Project funding (Div. I-III),Project funding,Institut für ökumenische Studien Université de...,Université de Fribourg - FR,10102,"Religious sciences, Theology",Human and Social Sciences;Theology & religious...,01.01.1976,31.12.1976,25403.00,NaN
7,10,Konfuzianische Kulturwerte in der sozialen Ent...,NaN,Deuchler Martina,Project funding (Div. I-III),Project funding,Ostasiatisches Seminar Universität Zürich,Universität Zürich - ZH,10301,History in general,Human and Social Sciences;Theology & religious...,01.10.1975,31.03.1977,47100.00,NaN
8,11,Edizione degli scritti di Aurelio de' Giorgi B...,NaN,Stäuble Antonio,Project funding (Div. I-III),Project funding,NaN,Université de Lausanne - LA,10502,Romance languages and literature,Human and Social Sciences;Linguistics and lite...,01.10.1975,31.03.1977,25814.00,NaN
9,13,La construction de nouveautés au sein des morp...,NaN,Piaget Jean,Project funding (Div. I-III),Project funding,Laboratoire de Didactique et Epistémologie des...,Université de Genève - GE,10105,Psychology,"Human and Social Sciences;Psychology, educatio...",01.10.1975,30.09.1978,360000.00,NaN


#### We first begin by looking at the "Approved Amount" filed, which contains the amount the is given to the university.

In [8]:

data['Approved Amount'].value_counts().head()

data not included in P3    10910
10000.00                     611
5000.00                      348
8000.00                      320
15000.00                     273
Name: Approved Amount, dtype: int64

We can see that for 1/6 of the funding there is no amount that is registered, 
therefore we can remove them from the database as we will not use them for the sum of grant money in each canton.

In [10]:
data_clean = data[data['Approved Amount'].isin(['data not included in P3']) == False]

By reading the [SNF](http://p3.snf.ch/Pages/DataAndDocumentation.aspx) page we also find that if the research is not carried out at a swiss institution, the 'University' field will be left blank. As we are not interested in funding that goes outside of Switzerland we can also delete these rows.

In [12]:
swissFunding = data_clean.dropna(subset=['University'])

As we can see that made us drop a large part of the data already.

In [13]:
print('Initial size of data : ', len(data))
print('Usable data for switzerland : ',len(swissFunding))
print('Percentage of Data kept',round(len(swissFunding)/len(data),3)*100,' %')


Initial size of data :  63969
Usable data for switzerland :  50878
Percentage of Data kept 79.5  %


For this analysis, we are only interested in the University name and the funding amount so that we can sort by cantons, we will therefore drop the other columns. 
We also change the type of the approved amount to float so that we can use them as numbers afterwards.

In [14]:
swissFunding = swissFunding[['University','Approved Amount']]
swissFunding['Approved Amount'] = swissFunding['Approved Amount'].astype(np.float)
swissFunding.dtypes

University          object
Approved Amount    float64
dtype: object

We then proceed to group by university.

In [17]:
uniGroup = swissFunding.groupby('University')

We then use the aggregate sum function to get the total funding per university.

In [18]:
fundingUni = uniGroup.sum()
fundingUni.head()

,Approved Amount
University,
AO Research Institute - AORI,3435621.0
Allergie- und Asthmaforschung - SIAF,19169965.0
Berner Fachhochschule - BFH,31028695.0
Biotechnologie Institut Thurgau - BITG,2492535.0
Centre de rech. sur l'environnement alpin - CREALP,1567678.0


In [19]:
fundingUni['Canton'] = ''
fundingUni.head()

,Approved Amount,Canton
University,,
AO Research Institute - AORI,3435621.0,
Allergie- und Asthmaforschung - SIAF,19169965.0,
Berner Fachhochschule - BFH,31028695.0,
Biotechnologie Institut Thurgau - BITG,2492535.0,
Centre de rech. sur l'environnement alpin - CREALP,1567678.0,


Definition of a function that will query the geonames API and return the name of a Canton if it appears in the search.


# Getting into the Details. Data extraction.

In [30]:
# Get  Shape of data set , split it, extract canton and return canton naame
# This uses function query terms defined in util.py

Test = []
for i in range(fundingUni.index.shape[0]):
    
    canton_name = 'not found'
    
    for term in fundingUni.index[i].split():
        result = utils.query_term(term)
        if result != 'no canton found':
            canton_name = result
            Test.append(result)

In [35]:
# from the cordinate function. We extract the coordinate
coordinates = utils.coordinate(Test)


In [40]:
#view of the coordinates returned. First 5 values
coordinates[1:5]

[[47.3685586, 8.5404434],
 [46.2303063, 7.6605757],
 [46.2303063, 7.6605757],
 [46.2303063, 7.6605757]]

In [51]:
# Sample
# This is needed for geocoded towork
url = 'https://maps.googleapis.com/maps/api/geocode/json'

g = geocoder.google([47.3685586, 8.5404434], method='reverse')

In [53]:
g

<[OK] Google - Reverse [Fraumünsterstrasse 13, 8001 Zürich, Switzerland]>

for item in coordinates:
    print (str(item[0]) + ',' + str(item[1]))

In [60]:
for item in coordinates:
    g = geocoder.google([str(item[0]), str(item[1])], method='reverse')
    print (g)

<[OK] Google - Reverse [Lehmgrube 2, 8566 Dotnacht, Switzerland]>
<[OK] Google - Reverse [Fraumünsterstrasse 13, 8001 Zürich, Switzerland]>
<[OK] Google - Reverse [Unnamed Road, 3948 Oberems, Switzerland]>
<[OK] Google - Reverse [Unnamed Road, 3948 Oberems, Switzerland]>
<[OK] Google - Reverse [Unnamed Road, 3948 Oberems, Switzerland]>
<[OK] Google - Reverse [Rue de Villars 11, 1307 Lussery-Villars, Switzerland]>
<[OK] Google - Reverse [Theaterpl., 6003 Luzern, Switzerland]>
<[OK] Google - Reverse [Unnamed Road, 39800 Besain, France]>
<[OK] Google - Reverse [Unnamed Road, 39800 Besain, France]>
<[OK] Google - Reverse [Unnamed Road, 3948 Oberems, Switzerland]>
<[OK] Google - Reverse [Via Principale 327, 7608 Bregaglia, Switzerland]>
<[OK] Google - Reverse [Fraumünsterstrasse 13, 8001 Zürich, Switzerland]>
<[OK] Google - Reverse [Marktgasse 19, 9000 St. Gallen, Switzerland]>
<[OK] Google - Reverse [Marktgasse 19, 9000 St. Gallen, Switzerland]>
<[OK] Google - Reverse [Place Pury 13, 2000 